In [1]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from torch.utils.data import DataLoader,Dataset
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score,roc_auc_score,accuracy_score
from tqdm import tqdm 
import json 
import os
import time
import numpy as np
from copy import deepcopy,copy

In [2]:
annos = json.load(open('./data_new/annos.json','r'))
keys=['label', 'subtlety', 'internalStructure', 'calcification', 'sphericity', 'margin', 'lobulation', 'spiculation', 'texture']
# keys2id =[5, 9, 15, 20, 25, 30, 35, 40]
# keys2id ={ 'subtlety': 5, 'internalStructure': 4, 'calcification': 6, 'sphericity': 5, 'margin': 5, 'lobulation': 5, 'spiculation': 5, 'texture': 5}
keys2id ={ 'subtlety': 0, 'internalStructure': 5, 
          'calcification': 9, 'sphericity': 15, 
          'margin': 20, 'lobulation': 25, 
          'spiculation': 30, 'texture': 35
         }

In [3]:
H = np.zeros((len(annos),40),dtype=int)
for lung in annos:
    row_id = int(lung)-1
    for key in annos[lung]:
        if key =='label':
            continue
        idx= keys2id[key]+ annos[lung][key]-1
        H[row_id][idx] = 1 

In [4]:
import scipy as sp
import scipy.sparse.linalg
from numpy import *
import numpy as np

def getBase(H_dense):
    K = 500
    tolerant = 0.1 ** 5
    epsilon = 0.1 ** 10
    patientNum, featureNum = len(H_dense), len(H_dense[0])
    H = sp.sparse.lil_matrix((patientNum, featureNum))
    D = sp.sparse.lil_matrix((patientNum, patientNum))
    D_edge = sp.sparse.lil_matrix((featureNum, featureNum))
    I = sp.sparse.lil_matrix(np.eye(patientNum, patientNum))

    # constructing the laplacian matrices
    print('Constructing the laplacian matrices...')
    for patient in range(patientNum):
        for feature in range(featureNum):
            if H_dense[patient][feature] > 0:
                H[patient, feature] = H_dense[patient][feature]
                D[patient, patient] += H_dense[patient][feature]
                D_edge[feature, feature] += H_dense[patient][feature]
    # print(H)
    D_n = sp.sparse.lil_matrix((patientNum, patientNum))
    D_e = sp.sparse.lil_matrix((featureNum, featureNum))
    for i in range(patientNum):
        D_n[i, i] = 1.0 / max(sqrt(D[i, i]), epsilon)
    for i in range(featureNum):
        D_e[i, i] = 1.0 / max(D_edge[i, i], epsilon)
    L = I - D_n * H * D_e * H.T * D_n

    #eigenvalue factorization
    print('Decomposing the laplacian matrices...')
    [Lamda, P] = sp.sparse.linalg.eigsh(L, k = K, which='SM', tol = tolerant)
    print(Lamda[0:10])
    return(P)

In [5]:
P=getBase(H.tolist())

Constructing the laplacian matrices...
Decomposing the laplacian matrices...
[1.41457433e-15 6.79548585e-01 7.31341624e-01 8.12738063e-01
 8.28526251e-01 8.39241781e-01 8.42142157e-01 8.46591347e-01
 8.50461969e-01 8.62346452e-01]


In [6]:
# 网络层
class GCNLayer(torch.nn.Module):
    def __init__(self,k_size,P,device):
        super().__init__()
        self.P=torch.tensor(P[:, 0: k_size]).double().to(device)
        self.k_size=k_size
        self.k =Parameter(torch.Tensor(k_size))
        self.reset_parameters()
    def reset_parameters(self):
        init.kaiming_uniform_(self.k.unsqueeze(0),a=math.sqrt(5))
    def forward(self,x):
        return self.P.mm(self.k.diag().mm(self.P.t().mm(x)))

In [7]:
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.init as init
import math

class gcn(torch.nn.Module):
    def __init__(self,in_features, out_features,k_size,layers ,P,device,alpha=[1,0.1,0.01,0.001]):
        super().__init__()
        self.layers= layers
        self.in_features=in_features
        self.out_features=out_features
        self.k_size=k_size
#         self.linear = nn.Linear(self.in_features,self.out_features*8)
        self.embeddings = nn.Embedding(self.in_features,self.out_features)
        self.gcnlayers = nn.ModuleList()
        for layer in range (self.layers):
            self.gcnlayers.append(GCNLayer(self.k_size,P,device))
        self.device = device
#         self.fc = nn.Linear((2048+self.out_features*8)*(self.layers+1),2)
        self.fc = nn.Linear((2048+self.out_features*8),2)
        self.alpha = alpha 
    def forward(self,x1,x2):
#         t_features= self.linear(x1)
        t_features= self.embeddings(x1)
        t_features= torch.reshape(t_features,(t_features.shape[0],-1))
        out= torch.cat((x2,t_features),dim=1)
        outputs =[out]
        
        for i,gcnlayer in enumerate(self.gcnlayers):
            out = torch.relu(gcnlayer(out))
#             out = out + self.alpha[i]*torch.relu(gcnlayer(out))
#             outputs += self.alpha[i] * out 
            outputs.append(out)
            
        out = sum([outputs[i]*self.alpha[i] for i in range(self.layers+1)])
        return (self.fc(out))
        

In [8]:
# config 
device = torch.device("cuda:0")

max_epoch=500
out_dir = 'result_out_gcn'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
best_accuracy = 1e-9
val_interval=1
# weight_decay=0.01
# gcn_layer_num = 1
# k_size = 200
# learning_rate=0.01

In [9]:
# label 
np_dir = './data'
random_state = 0
nps_labels = np.array([[os.path.join(np_dir, '{}.npy'.format(fig_id)), annos[fig_id]['label']] for fig_id in annos])
sss= StratifiedShuffleSplit(n_splits=1,test_size=0.3,random_state=random_state)
# train_index,val_index=next((sss.split(nps_labels,nps_labels[:,1])))
train_index,val_index=next((sss.split(nps_labels,nps_labels[:,1])))
# train_nps,val_nps=nps_labels[train_index,0],nps_labels[val_index,0]
# train_labels,val_labels= nps_labels[train_index,1].astype(np.int),nps_labels[val_index,1].astype(np.int)
train_val_labels=torch.from_numpy(nps_labels[:,1].astype(np.int)).long().to(device)

train_val_labels[train_val_labels==-1]=0
train_flag = torch.zeros(len(nps_labels)).long()
train_flag[train_index]=1

# data
x2= np.load('./features.npy')
torch_x2= torch.from_numpy(x2).double().to(device)
h=[]
for row in H:
    h.append([])
    for i in range(H.shape[1]):
        if row[i]:
            h[-1].append(i)
torch_x1= torch.LongTensor(h).to(device)
# torch_x1= torch.from_numpy(H).double().to(device)

In [28]:
record ={}
for weight_decay in [0.1,0.01,0.001,0]:
    record [weight_decay]={}
    for gcn_layer_num in [1,2,3]:
        record[weight_decay][gcn_layer_num]={}
        for learning_rate in [0.01,0.001]:
            record[weight_decay][gcn_layer_num][learning_rate]={}
            for k_size in [5,50,500]:# [50,100,150,200,250,300,350,400,450,500]:
                record[weight_decay][gcn_layer_num][learning_rate][k_size]=[0,0]
                
                # model
                model = gcn(40,32,k_size,gcn_layer_num,P,device)
                # model = Linear1(48,512,500,P,device)
                model.double().to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99),weight_decay=weight_decay)
                criterion = torch.nn.CrossEntropyLoss(reduction="none")
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=40, verbose=False,
                                                                       min_lr=1e-6)
                # training 
                model.train()
                for epoch in (range(max_epoch)):
                    optimizer.zero_grad()
                    start=time.time()
                    out = model(torch_x1,torch_x2)
                    loss = criterion(out, train_val_labels)
                    train_loss= loss[train_flag==1].mean()
                    val_loss= loss[train_flag==0].mean()
                    train_loss.backward()
                    #     print(train_loss)
                    optimizer.step()

                    predict = out[:,1].detach().to('cpu').numpy()
                    acc_predict= out.argmax(dim=1).detach().to('cpu').numpy()
                    np_train_labels = train_val_labels.detach().to('cpu').numpy()

                    auc_train= roc_auc_score(y_true=np_train_labels[train_flag.numpy()==1],y_score=predict[train_flag.numpy()==1])
                    auc_val =roc_auc_score(y_true=np_train_labels[train_flag.numpy()==0],y_score=predict[train_flag.numpy()==0] )

                    acc_train = accuracy_score(np_train_labels[train_flag.numpy()==1],acc_predict[train_flag.numpy()==1])
                    acc_val = accuracy_score(np_train_labels[train_flag.numpy()==0],acc_predict[train_flag.numpy()==0])

                    scheduler.step(auc_val)
                    #                     print('train: [%4d/ %5d], train loss: %.6f, val loss : %.6f, train auc: %.6f, val auc: %.6f, train acc: %.6f, val acc: %.6f,  time: %f s' %
                    #                               (epoch + 1, max_epoch, train_loss.item(), val_loss.item(), auc_train,auc_val,acc_train,acc_val, time.time() - start))
                    record[weight_decay][gcn_layer_num][learning_rate][k_size]=[max(record[weight_decay][gcn_layer_num][learning_rate][k_size][0],auc_val),
                                                                                max(record[weight_decay][gcn_layer_num][learning_rate][k_size][1],acc_val)]
                    
                    json.dump(record,open('parameter_select.json','w'))
                print ('weight_decay:{}, gcn_layer_num:{},k_size:{},learning_rate:{},auc:{},acc:{}'.format(weight_decay,gcn_layer_num,k_size,learning_rate,\
                                                                                                           record[weight_decay][gcn_layer_num][learning_rate][k_size][0],\
                                                                                                          record[weight_decay][gcn_layer_num][learning_rate][k_size][1]))

weight_decay:0.1, gcn_layer_num:1,k_size:5,learning_rate:0.01,auc:0.9308022756298617,acc:0.8829516539440203
weight_decay:0.1, gcn_layer_num:1,k_size:50,learning_rate:0.01,auc:0.9260130035992105,acc:0.8778625954198473
weight_decay:0.1, gcn_layer_num:1,k_size:500,learning_rate:0.01,auc:0.9312957157784744,acc:0.8854961832061069
weight_decay:0.1, gcn_layer_num:1,k_size:5,learning_rate:0.001,auc:0.9252293045396494,acc:0.8727735368956743
weight_decay:0.1, gcn_layer_num:1,k_size:50,learning_rate:0.001,auc:0.9258678741437363,acc:0.8651399491094147
weight_decay:0.1, gcn_layer_num:1,k_size:500,learning_rate:0.001,auc:0.9262161848368744,acc:0.8676844783715013
weight_decay:0.1, gcn_layer_num:2,k_size:5,learning_rate:0.01,auc:0.9278996865203761,acc:0.8778625954198473
weight_decay:0.1, gcn_layer_num:2,k_size:50,learning_rate:0.01,auc:0.920440032508998,acc:0.8702290076335878
weight_decay:0.1, gcn_layer_num:2,k_size:500,learning_rate:0.01,auc:0.924068268895855,acc:0.8625954198473282
weight_decay:0.1, 

In [41]:
record ={}
for weight_decay in [0.1,0.01,0.001,0]:
    record [weight_decay]={}
    for gcn_layer_num in [0,1,2,3]:
        record[weight_decay][gcn_layer_num]={}
        for learning_rate in [0.01,0.001]:
            record[weight_decay][gcn_layer_num][learning_rate]={}
            for k_size in [5,50,500]:# [50,100,150,200,250,300,350,400,450,500]:
                record[weight_decay][gcn_layer_num][learning_rate][k_size]=[0,0]
                
                # model
                model = gcn(40,32,k_size,gcn_layer_num,P,device)
                # model = Linear1(48,512,500,P,device)
                model.double().to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99),weight_decay=weight_decay)
                criterion = torch.nn.CrossEntropyLoss(reduction="none")
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=40, verbose=False,
                                                                       min_lr=1e-6)
                # training 
                model.train()
                for epoch in (range(max_epoch)):
                    optimizer.zero_grad()
                    start=time.time()
                    out = model(torch_x1,torch_x2)
                    loss = criterion(out, train_val_labels)
                    train_loss= loss[train_flag==1].mean()
                    val_loss= loss[train_flag==0].mean()
                    train_loss.backward()
                    #     print(train_loss)
                    optimizer.step()

                    predict = out[:,1].detach().to('cpu').numpy()
                    acc_predict= out.argmax(dim=1).detach().to('cpu').numpy()
                    np_train_labels = train_val_labels.detach().to('cpu').numpy()

                    auc_train= roc_auc_score(y_true=np_train_labels[train_flag.numpy()==1],y_score=predict[train_flag.numpy()==1])
                    auc_val =roc_auc_score(y_true=np_train_labels[train_flag.numpy()==0],y_score=predict[train_flag.numpy()==0] )

                    acc_train = accuracy_score(np_train_labels[train_flag.numpy()==1],acc_predict[train_flag.numpy()==1])
                    acc_val = accuracy_score(np_train_labels[train_flag.numpy()==0],acc_predict[train_flag.numpy()==0])

                    scheduler.step(auc_val)
                    #                     print('train: [%4d/ %5d], train loss: %.6f, val loss : %.6f, train auc: %.6f, val auc: %.6f, train acc: %.6f, val acc: %.6f,  time: %f s' %
                    #                               (epoch + 1, max_epoch, train_loss.item(), val_loss.item(), auc_train,auc_val,acc_train,acc_val, time.time() - start))
                    record[weight_decay][gcn_layer_num][learning_rate][k_size]=[max(record[weight_decay][gcn_layer_num][learning_rate][k_size][0],auc_val),
                                                                                max(record[weight_decay][gcn_layer_num][learning_rate][k_size][1],acc_val)]
                    
                    json.dump(record,open('parameter_select.json','w'))
                print ('weight_decay:{}, gcn_layer_num:{},k_size:{},learning_rate:{},auc:{},acc:{}'.format(weight_decay,gcn_layer_num,k_size,learning_rate,\
                                                                                                           record[weight_decay][gcn_layer_num][learning_rate][k_size][0],\
                                                                                                          record[weight_decay][gcn_layer_num][learning_rate][k_size][1]))

weight_decay:0.1, gcn_layer_num:0,k_size:5,learning_rate:0.01,auc:0.9287704632532219,acc:0.8651399491094147
weight_decay:0.1, gcn_layer_num:0,k_size:50,learning_rate:0.01,auc:0.9261871589457796,acc:0.8676844783715013
weight_decay:0.1, gcn_layer_num:0,k_size:500,learning_rate:0.01,auc:0.9255776152327877,acc:0.8753180661577609
weight_decay:0.1, gcn_layer_num:0,k_size:5,learning_rate:0.001,auc:0.9258098223615465,acc:0.8651399491094147
weight_decay:0.1, gcn_layer_num:0,k_size:50,learning_rate:0.001,auc:0.9232845698362939,acc:0.8727735368956743
weight_decay:0.1, gcn_layer_num:0,k_size:500,learning_rate:0.001,auc:0.9269708580053408,acc:0.8727735368956743
weight_decay:0.1, gcn_layer_num:1,k_size:5,learning_rate:0.01,auc:0.9301056542435853,acc:0.8753180661577609
weight_decay:0.1, gcn_layer_num:1,k_size:50,learning_rate:0.01,auc:0.9240102171136654,acc:0.8727735368956743
weight_decay:0.1, gcn_layer_num:1,k_size:500,learning_rate:0.01,auc:0.9328921397886916,acc:0.8676844783715013
weight_decay:0.1

In [10]:
record = [0,0]
weight_decay =0.01
gcn_layer_num =1
learning_rate =0.01
k_size = 50# [50,100,150,200,250,300,350,400,450,500]:
alphas=[[1,0.1,0.01,0.001],
        [1,0.2,0.01,0.001],
        [1,0.3,0.01,0.001],
        [1,0.4,0.01,0.001],
        [1,0.5,0.01,0.001],
        [1,0.6,0.01,0.001],
        [1,0.7,0.01,0.001],
        [1,0.8,0.01,0.001],
        [1,0.9,0.01,0.001],
        [1,1,0.01,0.001]]
for alpha in alphas :
    # model
    model = gcn(40,32,k_size,gcn_layer_num,P,device,alpha)
    # model = Linear1(48,512,500,P,device)
    model.double().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99),weight_decay=weight_decay)
    criterion = torch.nn.CrossEntropyLoss(reduction="none")
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=40, verbose=False,
                                                           min_lr=1e-6)
    # training 
    model.train()
    for epoch in (range(max_epoch)):
        optimizer.zero_grad()
        start=time.time()
        out = model(torch_x1,torch_x2)
        loss = criterion(out, train_val_labels)
        train_loss= loss[train_flag==1].mean()
        val_loss= loss[train_flag==0].mean()
        train_loss.backward()
        #     print(train_loss)
        optimizer.step()

        predict = out[:,1].detach().to('cpu').numpy()
        acc_predict= out.argmax(dim=1).detach().to('cpu').numpy()
        np_train_labels = train_val_labels.detach().to('cpu').numpy()

        auc_train= roc_auc_score(y_true=np_train_labels[train_flag.numpy()==1],y_score=predict[train_flag.numpy()==1])
        auc_val =roc_auc_score(y_true=np_train_labels[train_flag.numpy()==0],y_score=predict[train_flag.numpy()==0] )

        acc_train = accuracy_score(np_train_labels[train_flag.numpy()==1],acc_predict[train_flag.numpy()==1])
        acc_val = accuracy_score(np_train_labels[train_flag.numpy()==0],acc_predict[train_flag.numpy()==0])

        scheduler.step(auc_val)
        #                     print('train: [%4d/ %5d], train loss: %.6f, val loss : %.6f, train auc: %.6f, val auc: %.6f, train acc: %.6f, val acc: %.6f,  time: %f s' %
        #                               (epoch + 1, max_epoch, train_loss.item(), val_loss.item(), auc_train,auc_val,acc_train,acc_val, time.time() - start))
#         record[weight_decay][gcn_layer_num][learning_rate][k_size]=[max(record[weight_decay][gcn_layer_num][learning_rate][k_size][0],auc_val),
#                                                                     max(record[weight_decay][gcn_layer_num][learning_rate][k_size][1],acc_val)]
        record =[max(record[0],auc_val),max(record[1],acc_val)]

        json.dump(record,open('parameter_select.json','w'))
    print(alpha)
    print('auc:{},acc:{}'.format(record[0],record[1]))
#     print ('weight_decay:{}, gcn_layer_num:{},k_size:{},learning_rate:{},auc:{},acc:{}'.format(weight_decay,gcn_layer_num,k_size,learning_rate,\
#                                                                                                record[weight_decay][gcn_layer_num][learning_rate][k_size][0],\
#                                                                                               record[weight_decay][gcn_layer_num][learning_rate][k_size][1]))

[1, 0.1, 0.01, 0.001]
auc:0.9316150005805178,acc:0.8778625954198473
[1, 0.2, 0.01, 0.001]
auc:0.9316150005805178,acc:0.8778625954198473
[1, 0.3, 0.01, 0.001]
auc:0.9316150005805178,acc:0.8778625954198473
[1, 0.4, 0.01, 0.001]
auc:0.9316150005805178,acc:0.8778625954198473
[1, 0.5, 0.01, 0.001]
auc:0.9316150005805178,acc:0.8778625954198473
[1, 0.6, 0.01, 0.001]
auc:0.9316150005805178,acc:0.8804071246819338
[1, 0.7, 0.01, 0.001]
auc:0.9316150005805178,acc:0.8804071246819338
[1, 0.8, 0.01, 0.001]
auc:0.9316150005805178,acc:0.8804071246819338
[1, 0.9, 0.01, 0.001]
auc:0.9316150005805178,acc:0.8804071246819338
[1, 1, 0.01, 0.001]
auc:0.9317020782538024,acc:0.8804071246819338


In [10]:
record = [0,0]
weight_decay = 0.001
gcn_layer_num =2
learning_rate =0.01
k_size = 50# [50,100,150,200,250,300,350,400,450,500]:
alphas=[[1,0.1,0.1,0.001],
        [1,0.2,0.2,0.001],
        [1,0.3,0.3,0.001],
        [1,0.4,0.4,0.001],
        [1,0.5,0.5,0.001],
        [1,0.6,0.6,0.001],
        [1,0.7,0.7,0.001],
        [1,0.8,0.8,0.001],
        [1,0.9,0.9,0.001],
        [1,1,1,0.001]]
for alpha in alphas :
    # model
    model = gcn(40,32,k_size,gcn_layer_num,P,device,alpha)
    # model = Linear1(48,512,500,P,device)
    model.double().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99),weight_decay=weight_decay)
    criterion = torch.nn.CrossEntropyLoss(reduction="none")
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=40, verbose=False,
                                                           min_lr=1e-6)
    # training 
    model.train()
    for epoch in (range(max_epoch)):
        optimizer.zero_grad()
        start=time.time()
        out = model(torch_x1,torch_x2)
        loss = criterion(out, train_val_labels)
        train_loss= loss[train_flag==1].mean()
        val_loss= loss[train_flag==0].mean()
        train_loss.backward()
        #     print(train_loss)
        optimizer.step()

        predict = out[:,1].detach().to('cpu').numpy()
        acc_predict= out.argmax(dim=1).detach().to('cpu').numpy()
        np_train_labels = train_val_labels.detach().to('cpu').numpy()

        auc_train= roc_auc_score(y_true=np_train_labels[train_flag.numpy()==1],y_score=predict[train_flag.numpy()==1])
        auc_val =roc_auc_score(y_true=np_train_labels[train_flag.numpy()==0],y_score=predict[train_flag.numpy()==0] )

        acc_train = accuracy_score(np_train_labels[train_flag.numpy()==1],acc_predict[train_flag.numpy()==1])
        acc_val = accuracy_score(np_train_labels[train_flag.numpy()==0],acc_predict[train_flag.numpy()==0])

        scheduler.step(auc_val)
        #                     print('train: [%4d/ %5d], train loss: %.6f, val loss : %.6f, train auc: %.6f, val auc: %.6f, train acc: %.6f, val acc: %.6f,  time: %f s' %
        #                               (epoch + 1, max_epoch, train_loss.item(), val_loss.item(), auc_train,auc_val,acc_train,acc_val, time.time() - start))
#         record[weight_decay][gcn_layer_num][learning_rate][k_size]=[max(record[weight_decay][gcn_layer_num][learning_rate][k_size][0],auc_val),
#                                                                     max(record[weight_decay][gcn_layer_num][learning_rate][k_size][1],acc_val)]
        record =[max(record[0],auc_val),max(record[1],acc_val)]

        json.dump(record,open('parameter_select.json','w'))
    print(alpha)
    print('auc:{},acc:{}'.format(record[0],record[1]))
#     print ('weight_decay:{}, gcn_layer_num:{},k_size:{},learning_rate:{},auc:{},acc:{}'.format(weight_decay,gcn_layer_num,k_size,learning_rate,\
#                                                                                                record[weight_decay][gcn_layer_num][learning_rate][k_size][0],\
#                                                                                               record[weight_decay][gcn_layer_num][learning_rate][k_size][1]))

[1, 0.1, 0.1, 0.001]
auc:0.9292348775107396,acc:0.8778625954198473
[1, 0.2, 0.2, 0.001]
auc:0.9306571461743874,acc:0.8778625954198473
[1, 0.3, 0.3, 0.001]
auc:0.9306571461743874,acc:0.8778625954198473
[1, 0.4, 0.4, 0.001]
auc:0.9306571461743874,acc:0.8778625954198473
[1, 0.5, 0.5, 0.001]
auc:0.9306571461743874,acc:0.8778625954198473
[1, 0.6, 0.6, 0.001]
auc:0.9306571461743874,acc:0.8778625954198473
[1, 0.7, 0.7, 0.001]
auc:0.9306571461743874,acc:0.8778625954198473
[1, 0.8, 0.8, 0.001]
auc:0.9306571461743874,acc:0.8778625954198473
[1, 0.9, 0.9, 0.001]
auc:0.9306571461743874,acc:0.8778625954198473
[1, 1, 1, 0.001]
auc:0.9306571461743874,acc:0.8778625954198473


In [11]:
record = [0,0]
weight_decay = 0.001
gcn_layer_num =3
learning_rate =0.01
k_size = 50# [50,100,150,200,250,300,350,400,450,500]:
alphas=[[1,0.1,0.1,0.1],
        [1,0.2,0.2,0.2],
        [1,0.3,0.3,0.3],
        [1,0.4,0.4,0.4],
        [1,0.5,0.5,0.5],
        [1,0.6,0.6,0.6],
        [1,0.7,0.7,0.7],
        [1,0.8,0.8,0.8],
        [1,0.9,0.9,0.9],
        [1,1,1,1]]
for alpha in alphas :
    # model
    model = gcn(40,32,k_size,gcn_layer_num,P,device,alpha)
    # model = Linear1(48,512,500,P,device)
    model.double().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99),weight_decay=weight_decay)
    criterion = torch.nn.CrossEntropyLoss(reduction="none")
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=40, verbose=False,
                                                           min_lr=1e-6)
    # training 
    model.train()
    for epoch in (range(max_epoch)):
        optimizer.zero_grad()
        start=time.time()
        out = model(torch_x1,torch_x2)
        loss = criterion(out, train_val_labels)
        train_loss= loss[train_flag==1].mean()
        val_loss= loss[train_flag==0].mean()
        train_loss.backward()
        #     print(train_loss)
        optimizer.step()

        predict = out[:,1].detach().to('cpu').numpy()
        acc_predict= out.argmax(dim=1).detach().to('cpu').numpy()
        np_train_labels = train_val_labels.detach().to('cpu').numpy()

        auc_train= roc_auc_score(y_true=np_train_labels[train_flag.numpy()==1],y_score=predict[train_flag.numpy()==1])
        auc_val =roc_auc_score(y_true=np_train_labels[train_flag.numpy()==0],y_score=predict[train_flag.numpy()==0] )

        acc_train = accuracy_score(np_train_labels[train_flag.numpy()==1],acc_predict[train_flag.numpy()==1])
        acc_val = accuracy_score(np_train_labels[train_flag.numpy()==0],acc_predict[train_flag.numpy()==0])

        scheduler.step(auc_val)
        #                     print('train: [%4d/ %5d], train loss: %.6f, val loss : %.6f, train auc: %.6f, val auc: %.6f, train acc: %.6f, val acc: %.6f,  time: %f s' %
        #                               (epoch + 1, max_epoch, train_loss.item(), val_loss.item(), auc_train,auc_val,acc_train,acc_val, time.time() - start))
#         record[weight_decay][gcn_layer_num][learning_rate][k_size]=[max(record[weight_decay][gcn_layer_num][learning_rate][k_size][0],auc_val),
#                                                                     max(record[weight_decay][gcn_layer_num][learning_rate][k_size][1],acc_val)]
        record =[max(record[0],auc_val),max(record[1],acc_val)]

        json.dump(record,open('parameter_select.json','w'))
    print(alpha)
    print('auc:{},acc:{}'.format(record[0],record[1]))
#     print ('weight_decay:{}, gcn_layer_num:{},k_size:{},learning_rate:{},auc:{},acc:{}'.format(weight_decay,gcn_layer_num,k_size,learning_rate,\
#                                                                                                record[weight_decay][gcn_layer_num][learning_rate][k_size][0],\
#                                                                                               record[weight_decay][gcn_layer_num][learning_rate][k_size][1]))

[1, 0.1, 0.1, 0.1]
auc:0.9330662951352607,acc:0.8753180661577609
[1, 0.2, 0.2, 0.2]
auc:0.9330662951352607,acc:0.8778625954198473
[1, 0.3, 0.3, 0.3]
auc:0.9330662951352607,acc:0.8778625954198473
[1, 0.4, 0.4, 0.4]
auc:0.9330662951352607,acc:0.8854961832061069
[1, 0.5, 0.5, 0.5]
auc:0.9330662951352607,acc:0.8854961832061069
[1, 0.6, 0.6, 0.6]
auc:0.9330662951352607,acc:0.8854961832061069
[1, 0.7, 0.7, 0.7]
auc:0.9330662951352607,acc:0.8854961832061069
[1, 0.8, 0.8, 0.8]
auc:0.9330662951352607,acc:0.8854961832061069
[1, 0.9, 0.9, 0.9]
auc:0.9330662951352607,acc:0.8854961832061069
[1, 1, 1, 1]
auc:0.9330662951352607,acc:0.8854961832061069


In [13]:
record = [0,0]
weight_decay = 0.001
gcn_layer_num =3
learning_rate =0.01
k_size = 50# [50,100,150,200,250,300,350,400,450,500]:
alphas=[[1,0.1,0.1,0.1],
        [1,0.2,0.2,0.2],
        [1,0.3,0.3,0.3],
        [1,0.4,0.4,0.4],
        [1,0.5,0.5,0.5],
        [1,0.6,0.6,0.6],
        [1,0.7,0.7,0.7],
        [1,0.8,0.8,0.8],
        [1,0.9,0.9,0.9],
        [1,1,1,1]]
for alpha in alphas :
    # model
    model = gcn(40,32,k_size,gcn_layer_num,P,device,alpha)
    # model = Linear1(48,512,500,P,device)
    model.double().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99),weight_decay=weight_decay)
    criterion = torch.nn.CrossEntropyLoss(reduction="none")
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=40, verbose=False,
                                                           min_lr=1e-6)
    # training 
    model.train()
    print(model.alpha)
    for epoch in (range(max_epoch)):
        optimizer.zero_grad()
        start=time.time()
        out = model(torch_x1,torch_x2)
        loss = criterion(out, train_val_labels)
        train_loss= loss[train_flag==1].mean()
        val_loss= loss[train_flag==0].mean()
        train_loss.backward()
        #     print(train_loss)
        optimizer.step()

        predict = out[:,1].detach().to('cpu').numpy()
        acc_predict= out.argmax(dim=1).detach().to('cpu').numpy()
        np_train_labels = train_val_labels.detach().to('cpu').numpy()

        auc_train= roc_auc_score(y_true=np_train_labels[train_flag.numpy()==1],y_score=predict[train_flag.numpy()==1])
        auc_val =roc_auc_score(y_true=np_train_labels[train_flag.numpy()==0],y_score=predict[train_flag.numpy()==0] )

        acc_train = accuracy_score(np_train_labels[train_flag.numpy()==1],acc_predict[train_flag.numpy()==1])
        acc_val = accuracy_score(np_train_labels[train_flag.numpy()==0],acc_predict[train_flag.numpy()==0])

        scheduler.step(auc_val)
        #                     print('train: [%4d/ %5d], train loss: %.6f, val loss : %.6f, train auc: %.6f, val auc: %.6f, train acc: %.6f, val acc: %.6f,  time: %f s' %
        #                               (epoch + 1, max_epoch, train_loss.item(), val_loss.item(), auc_train,auc_val,acc_train,acc_val, time.time() - start))
#         record[weight_decay][gcn_layer_num][learning_rate][k_size]=[max(record[weight_decay][gcn_layer_num][learning_rate][k_size][0],auc_val),
#                                                                     max(record[weight_decay][gcn_layer_num][learning_rate][k_size][1],acc_val)]
        record =[max(record[0],auc_val),max(record[1],acc_val)]

        json.dump(record,open('parameter_select.json','w'))
    
    print('auc:{},acc:{}'.format(record[0],record[1]))
#     print ('weight_decay:{}, gcn_layer_num:{},k_size:{},learning_rate:{},auc:{},acc:{}'.format(weight_decay,gcn_layer_num,k_size,learning_rate,\
#                                                                                                record[weight_decay][gcn_layer_num][learning_rate][k_size][0],\
#                                                                                               record[weight_decay][gcn_layer_num][learning_rate][k_size][1]))

[1, 0.1, 0.1, 0.1]
auc:0.9322245442935098,acc:0.8778625954198473
[1, 0.2, 0.2, 0.2]
auc:0.9322245442935098,acc:0.8778625954198473
[1, 0.3, 0.3, 0.3]


KeyboardInterrupt: 

In [14]:
record = [0,0]
weight_decay = 0.001
gcn_layer_num =0
learning_rate =0.01
k_size = 50# [50,100,150,200,250,300,350,400,450,500]:
alphas=[[1,0.1,0.1,0.1],
        [1,0.2,0.2,0.2],
        [1,0.3,0.3,0.3],
        [1,0.4,0.4,0.4],
        [1,0.5,0.5,0.5],
        [1,0.6,0.6,0.6],
        [1,0.7,0.7,0.7],
        [1,0.8,0.8,0.8],
        [1,0.9,0.9,0.9],
        [1,1,1,1]]
for alpha in alphas :
    # model
    model = gcn(40,32,k_size,gcn_layer_num,P,device,alpha)
    # model = Linear1(48,512,500,P,device)
    model.double().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99),weight_decay=weight_decay)
    criterion = torch.nn.CrossEntropyLoss(reduction="none")
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=40, verbose=False,
                                                           min_lr=1e-6)
    # training 
    model.train()
    print(model.alpha)
    for epoch in (range(max_epoch)):
        optimizer.zero_grad()
        start=time.time()
        out = model(torch_x1,torch_x2)
        loss = criterion(out, train_val_labels)
        train_loss= loss[train_flag==1].mean()
        val_loss= loss[train_flag==0].mean()
        train_loss.backward()
        #     print(train_loss)
        optimizer.step()

        predict = out[:,1].detach().to('cpu').numpy()
        acc_predict= out.argmax(dim=1).detach().to('cpu').numpy()
        np_train_labels = train_val_labels.detach().to('cpu').numpy()

        auc_train= roc_auc_score(y_true=np_train_labels[train_flag.numpy()==1],y_score=predict[train_flag.numpy()==1])
        auc_val =roc_auc_score(y_true=np_train_labels[train_flag.numpy()==0],y_score=predict[train_flag.numpy()==0] )

        acc_train = accuracy_score(np_train_labels[train_flag.numpy()==1],acc_predict[train_flag.numpy()==1])
        acc_val = accuracy_score(np_train_labels[train_flag.numpy()==0],acc_predict[train_flag.numpy()==0])

        scheduler.step(auc_val)
        #                     print('train: [%4d/ %5d], train loss: %.6f, val loss : %.6f, train auc: %.6f, val auc: %.6f, train acc: %.6f, val acc: %.6f,  time: %f s' %
        #                               (epoch + 1, max_epoch, train_loss.item(), val_loss.item(), auc_train,auc_val,acc_train,acc_val, time.time() - start))
#         record[weight_decay][gcn_layer_num][learning_rate][k_size]=[max(record[weight_decay][gcn_layer_num][learning_rate][k_size][0],auc_val),
#                                                                     max(record[weight_decay][gcn_layer_num][learning_rate][k_size][1],acc_val)]
        record =[max(record[0],auc_val),max(record[1],acc_val)]

        json.dump(record,open('parameter_select.json','w'))
    
    print('auc:{},acc:{}'.format(record[0],record[1]))
#     print ('weight_decay:{}, gcn_layer_num:{},k_size:{},learning_rate:{},auc:{},acc:{}'.format(weight_decay,gcn_layer_num,k_size,learning_rate,\
#                                                                                                record[weight_decay][gcn_layer_num][learning_rate][k_size][0],\
#                                                                                               record[weight_decay][gcn_layer_num][learning_rate][k_size][1]))

[1, 0.1, 0.1, 0.1]
auc:0.9292058516196448,acc:0.8676844783715013
[1, 0.2, 0.2, 0.2]
auc:0.9292058516196448,acc:0.8778625954198473
[1, 0.3, 0.3, 0.3]
auc:0.9292058516196448,acc:0.8778625954198473
[1, 0.4, 0.4, 0.4]
auc:0.9302507836990596,acc:0.8804071246819338
[1, 0.5, 0.5, 0.5]
auc:0.9302507836990596,acc:0.8854961832061069
[1, 0.6, 0.6, 0.6]
auc:0.9302507836990596,acc:0.8854961832061069
[1, 0.7, 0.7, 0.7]
auc:0.9302507836990596,acc:0.8854961832061069
[1, 0.8, 0.8, 0.8]
auc:0.9302507836990596,acc:0.8854961832061069
[1, 0.9, 0.9, 0.9]
auc:0.9302507836990596,acc:0.8854961832061069
[1, 1, 1, 1]
auc:0.9302507836990596,acc:0.8854961832061069
